# データの準備及びDoc2Vecモデルの訓練

ライブラリーと変数の宣言

In [1]:
from ast_draw import CodeTransformer, counter
import glob
import node_to_vec as n2v
import ast

import networkx as nx
import csv
import os

In [2]:
pyfile = glob.glob('/Users/wintor7/Research/train_data_file/**/*.py', recursive=True)
var, for_var, func_var, func_params, func_return = {}, {}, {}, {}, {}
dict_func, var_counter = {},{}
node_to_id = {}
id_to_node = {}
node_to_id, id_to_node = n2v.create_node_lib()      #create node corpus
var_to_id, func_to_id = {},{}
id_to_var, id_to_func = {},{}
graphs = []


In [8]:
import random

pyfile = random.sample(pyfile, int(len(pyfile)/10))
len(pyfile)

4935

In [3]:
pyfile = glob.glob('/Users/wintor7/report7/report7_*/**/*.py')

In [4]:
len(pyfile)

224

ASTの抽出

In [6]:
for file in pyfile:
  try:
    with open(file, 'r') as fh:
      content = fh.read()

      #抽取AST
    r_node = ast.parse(content)

    transformer = CodeTransformer()
    r_node = transformer.visit(r_node)
    dict_func = counter(dict_func, transformer.var_in_func)



    func_list = list(transformer.var_in_func.keys())
    var_list = []
    for i,v in enumerate(transformer.var_in_func):
      for i in transformer.var_in_func[v]:
        var_list.append(i)

    name_line = transformer.name_line


    temp = {}
    for key in dict_func.keys():        #确认是否为常见变量名称
      for var in dict_func[key].keys():
        if var in name_line[key].keys():
          temp[var] = name_line[key][var]
        else:
          continue

    name_line = temp  
    node_list = {}
    for key in name_line.keys():          #
      for i in name_line[key]:
        transformer.track_line(r_node, i)
        if key not in node_list.keys():
          node_list[key] = [transformer.target_var]
        elif transformer.target_var not in node_list[key]:
          node_list[key].append(transformer.target_var)
    print(node_list)
    loop_list = {}
    for key in name_line.keys():          #
      for i in name_line[key]:
        transformer.track_loop(r_node,i)
        if key not in loop_list.keys():
          loop_list[key] = [transformer.target_loop]
        elif transformer.target_var not in loop_list[key]:
          loop_list[key].append(transformer.target_loop)


    def_list = {}
    for key in name_line.keys():          #
      for i in name_line[key]:
        transformer.track_def(r_node,i)
        if key not in def_list.keys():
          def_list[key] = [transformer.target_def]
        elif transformer.target_var not in def_list[key]:
          def_list[key].append(transformer.target_def)


    node_to_id, id_to_node = n2v.create_node_lib() 
    var_to_id, id_to_var = n2v.name_corpus(var_list, var_to_id, id_to_var)
    func_to_id, id_to_func = n2v.name_corpus(func_list, func_to_id, id_to_func)

    var_in_func = transformer.var_in_func
    #print('var_in_func: ', var_in_func)
    num = 0
    for func in var_in_func.keys():
      label1 = func
      for var in var_in_func[func]:
        num += 1
        label2 = var
        graph = nx.DiGraph(label1 = label1, label2 = label2)
        str_to_node = {}
        try:
          for var_node in node_list[var]:
            root = len(str_to_node)
            n2v.create_graph(var_node, node_to_id, graph, str_to_node, root)
            for node in graph.nodes():
              graph.nodes[node]['feature'] = str_to_node[node]
            root += 1
          nx.write_gpickle(graph,'/Users/wintor7/report7/graph/var/' + str(num) + '.gpickle')  
        except KeyError:
          continue

        graph_loop = nx.DiGraph(label1 = label1, label2 = label2)
        str_to_node = {}
        try:
          for var_node in loop_list[var]:
            root = len(str_to_node)
            n2v.create_graph(var_node, node_to_id, graph_loop, str_to_node, root)
            for node in graph_loop.nodes():
              graph_loop.nodes[node]['feature'] = str_to_node[node]
            root += 1
          nx.write_gpickle(graph_loop,'/Users/wintor7/report7/graph/loop/' + str(num) + '.gpickle')  
        except KeyError:
          continue


        graph_def = nx.DiGraph(label1 = label1, label2 = label2)
        str_to_node = {}
        try:
          for var_node in def_list[var]:
            root = len(str_to_node)
            n2v.create_graph(var_node, node_to_id, graph_def, str_to_node, root)
            for node in graph_def.nodes():
              graph_def.nodes[node]['feature'] = str_to_node[node]
            root += 1
          nx.write_gpickle(graph_def,'/Users/wintor7/report7/graph/def/' + str(num) + '.gpickle')  
        except KeyError:
          continue
  except:
    print(file)
    continue

/Users/wintor7/report7/report7_2021/e215735/report7-2.py
/Users/wintor7/report7/report7_2021/e215735/report7.py
/Users/wintor7/report7/report7_2021/e215732/report7.py
/Users/wintor7/report7/report7_2021/e215759/report7.py
/Users/wintor7/report7/report7_2021/e215750/report7.py
/Users/wintor7/report7/report7_2021/e215733/report7.py
/Users/wintor7/report7/report7_2021/e215705/report7.py
/Users/wintor7/report7/report7_2021/e215751/report7.py
/Users/wintor7/report7/report7_2021/e215756/report7.py
/Users/wintor7/report7/report7_2021/e215760/report7.py
/Users/wintor7/report7/report7_2021/e215740D/report7.2.py
/Users/wintor7/report7/report7_2021/e205237/report7.py
/Users/wintor7/report7/report7_2021/e205236/python_7.py
/Users/wintor7/report7/report7_2021/e215745/python_7.py
/Users/wintor7/report7/report7_2021/e205711/report7.py
/Users/wintor7/report7/report7_2021/e205515/report7.py
/Users/wintor7/report7/report7_2021/e215718/report7.py
/Users/wintor7/report7/report7_2021/e215718/report.py
/Use

モデルの宣言

In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from MyWeisfeilerLehaman import WeisfeilerLehmanHashing
from Mygraph2vec import Graph2Vec

データの前処理

In [3]:
import glob
import networkx as nx
deffile = glob.glob('/Users/wintor7/Research/defset/*.gpickle', recursive=True)
loopfile = glob.glob('/Users/wintor7/Research/loopset/*.gpickle', recursive=True)
graphfile = glob.glob('/Users/wintor7/Research/graphset_new/*.gpickle', recursive=True)

graphs = []
for i in range(int(len(graphfile)/10)):
  G = nx.read_gpickle(graphfile[i])
  graphs.append(G)
loop_graphs = []
for i in range(int(len(loopfile)/10)):
  G = nx.read_gpickle(loopfile[i])
  loop_graphs.append(G)
def_graphs = []
for i in range(int(len(deffile)/10)):
  G = nx.read_gpickle(deffile[i])
  def_graphs.append(G)

In [4]:
graphfileset = glob.glob("/Users/wintor7/report7/graph/var/*.gpickle")
loopfileset = glob.glob("/Users/wintor7/report7/graph/loop/*.gpickle")
deffileset = glob.glob("/Users/wintor7/report7/graph/def/*.gpickle")


In [5]:
for i in range(int(len(graphfileset))):
  G = nx.read_gpickle(graphfileset[i])
  graphs.append(G)
for i in range(int(len(loopfileset))):
  G = nx.read_gpickle(loopfileset[i])
  loop_graphs.append(G)
for i in range(len(deffileset)):
  G = nx.read_gpickle(deffileset[i])
  def_graphs.append(G)

In [6]:
len(graphs)

9993

In [7]:
graphs[8981].graph

{'label1': 'analyze', 'label2': 'length'}

モデルの訓練

In [8]:
model = Graph2Vec(attributed= True,dimensions=130, epochs= 25, learning_rate=0.015, mode=2)

In [9]:
model.fit(graphs, '/Users/wintor7/Research/model/new_model.model',loop_graphs=loop_graphs, def_graphs=def_graphs)

9993
{'label1': 'test_reflection', 'label2': 'test_reflect_key_name'}
{'label1': 'from_pretrained', 'label2': 'model_card_file'}
{'label1': 'revoke_data_sponsorship', 'label2': 'Optional'}
{'label1': 'CreateWidgets', 'label2': 'W'}
{'label1': 'revoke_claimable_balance_sponsorship', 'label2': 'Optional'}
{'label1': 'handler2', 'label2': 's'}
{'label1': 'testSetIdAttributeNS', 'label2': 'NS2'}
{'label1': 'test_then_values_2', 'label2': 'ListType'}
{'label1': 'test_romanize_royin_consistency', 'label2': 'romanize'}
{'label1': 'super', 'label2': 'name'}
{'label1': 'testSetIdAttributeNode', 'label2': 'NS2'}
{'label1': 'rebalance', 'label2': 'right_child'}
{'label1': 'close', 'label2': 'h'}
{'label1': 'dumpspecs', 'label2': 'os'}
{'label1': 'test_badandgoodsurrogatepassexceptions', 'label2': 'n'}
9978
['13', '13 39 59', '13 39 59 39 None 59 66 76', '59', '59 66 76', '59 66 76 66 75 76 None', '66', '66 75', '66 75 75 None', '75', '75 None', '75 None None', '76', '76 None', '76 None None', '39